# first attempt - simple reverse prediction

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
SIZE = 5
AMOUNT_BOARDS = 10000
AMOUNT_MOVES = 100
NUM_DICT = 1

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
print(name_df)

C:\GameOfLifeFiles\df\\5-10000\5size_10000boards_2gen_reverse


In [5]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [6]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 29760
reverse df sort: 29760


In [7]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  26784
len x test:  2976
len y train:  26784
len y test:  2976
17907 25


,train,test
accuracy,0.969795,0.638777
precision,0.990235,0.489044
recall,0.920595,0.466287
f1,0.954146,0.477394


In [8]:
reverse_df_sort.iloc[17].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0])

In [9]:
features.iloc[17].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [10]:
target.iloc[17]

0

In [11]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\reverse_model1\\{SIZE}\\decision_tree_size{SIZE}_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.683206  0.672715
precision  0.554524  0.557164
recall     0.365963  0.365622
f1         0.440930  0.441514
63 5
              train      test
accuracy   0.686417  0.672715
precision  0.561732  0.556671
recall     0.370229  0.368471
f1         0.446305  0.443429
127 6
              train      test
accuracy   0.690972  0.668683
precision  0.593521  0.564299
recall     0.300558  0.279202
f1         0.399042  0.373571
253 7
              train      test
accuracy   0.696274  0.671035
precision  0.600921  0.563793
recall     0.328229  0.310541
f1         0.424560  0.400490
503 8
              train      test
accuracy   0.705907  0.668683
precision  0.630033  0.559928
recall     0.335448  0.297246
f1         0.437799  0.388337
993 9
              train      test
accuracy   0.718825  0.669691
precision  0.663954  0.561837
recall     0.356994  0.301994
f1         0.464329  0.392835
1911 10
              train      test
accuracy   0.738575  0.66498

In [12]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 10, 42)
    name_file = f"{PATH_MODELS}\\reverse1\\{SIZE}\\dt_size{SIZE}_r1_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
1889 10
              train      test
accuracy   0.738127  0.672715
precision  0.696333  0.501563
recall     0.424295  0.328893
f1         0.527295  0.397277
1
1921 10
              train      test
accuracy   0.734954  0.681452
precision  0.678606  0.554252
recall     0.446946  0.369863
f1         0.538936  0.443662
2
1891 10
              train      test
accuracy   0.740255  0.673723
precision  0.681966  0.525741
recall     0.439714  0.335657
f1         0.534680  0.409726
3
1911 10
              train      test
accuracy   0.737306  0.684812
precision  0.689607  0.546341
recall     0.428804  0.337688
f1         0.528797  0.417391
4
1901 10
              train      test
accuracy   0.736074  0.688844
precision  0.691433  0.568000
recall     0.431755  0.351137
f1         0.531575  0.433985
5
1943 10
              train      test
accuracy   0.735103  0.670699
precision  0.689122  0.526555
recall     0.442203  0.341872
f1         0.538717  0.414576
6
1913 10
              train      test


KeyError: 'Col_50'

In [ ]:
X_test

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25
4827,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
17922,0,1,0,1,1,1,1,1,0,1,...,0,0,1,1,1,0,0,1,1,1
17197,0,1,0,1,0,0,1,0,1,0,...,1,1,1,0,0,1,1,0,0,0
5373,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,1,0,0,1,1,1
2632,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12395,0,0,1,0,1,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
15187,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
1518,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
8394,0,0,0,0,1,1,0,1,1,0,...,1,1,0,1,0,0,1,0,0,1


In [ ]:
PATH_DATA_TEST = 'C:\\GameOfLifeFiles\\dataTest\\'

In [ ]:
name_file = f"{PATH_DATA_TEST}\\data_test_size_{SIZE}.pkl"
with open(name_file, 'wb') as file:
    pickle.dump(X_test, file)